# FACEBOOK ADS

#### For more information you can check the link below:
#### https://developers.facebook.com/docs/marketing-api/targeting-search/

In [1]:
pip install facebook-business

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
# -*- coding: utf-8 -*-
__author__ = "Carolina Coimbra"

from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.targetingsearch import TargetingSearch
from facebook_business.api import FacebookAdsApi
from facebook_business.exceptions import FacebookError
from unicodedata import normalize

import numpy as np
import matplotlib.pyplot as plt
import string
import pandas as pd
import time

## FUNCTIONS TO MAKE REQUESTS

In [3]:
# Initialize the account

# PLEASE, UPDATE THE VERSION IF NECESSARY!

def get_ad_account(token, act_id, secret):

    if secret != '-':
        facebook_api = FacebookAdsApi.init(access_token=token, app_secret=secret, api_version='v16.0') #version
    else:
        facebook_api = FacebookAdsApi.init(access_token=token, api_version='v16.0') #version

    account = AdAccount('act_' + act_id)
    
    return facebook_api, account

In [4]:
# Return the audience for a specific target
def make_request(account, targeting_spec):
    api_params = {
        'targeting_spec': targeting_spec
    }
    
    reach_estimate = account.get_reach_estimate(params=api_params)
    number = reach_estimate[0]['users_lower_bound']
    #number = reach_estimate[0]['users_upper_bound']
    return number

In [5]:
# Return the interest id
def getInterestIDFromText(api, text, limit=10):
    params = {
        'q': text,        
        'type': 'adinterest',
        'limit': limit,        
    }
    
    resp = TargetingSearch.search(params=params, api=api)
    return resp

In [6]:
# Return the location key
def getLocationElement(api, element, location_type="country"):   
    params = {
        'q': element,
        'type': 'adgeolocation',
#       'location_types': ['city'] OR ['region'] OR ['country'] OR ['country_group'],
        'location_types': [location_type],
#       'countries': ['US']
    }
    
    responses = TargetingSearch.search(params=params, api=api)
    return responses

In [7]:
# Return a dictionary with the informations abount the interests
def testInterestSearch(api, list_of_interests_to_search): 
    
    print("interest_id \t name \t audience_size \t type")
       
    interests_dict = dict()

    for interest in list_of_interests_to_search:
        search_result = getInterestIDFromText(api, interest)
         
        for element in search_result: 
            #Consider just if the names is exactly the same!
            if interest.lower() == element["name"].lower():
                e_id = element["id"]
                e_name = interest
                e_audience = element["audience_size_lower_bound"]
                #e_audience = element["audience_size_upper_bound"]
                
                #not all responses have a topic and category defined. 
                try:
                    e_type = element["topic"]
                except:
                    e_type = ""
                interests_dict[e_name] = {"id": e_id, "audience": e_audience, "type": e_type}

                print (str(e_id) + "\t" + str(e_name) + "\t" + str(e_audience) + "\t" + str(e_type))
            
                break
          
    return interests_dict


In [8]:
# Return a dictionary with the informations abount the locations
def testSearchForLocationCode(api, list_of_locations_and_types_to_search):
    #available location types: country, country_group, region, city,

    print("key \t name \t location_type")
    
    locations_dict = dict()
    
    for location, l_type in list_of_locations_and_types_to_search:
        search_result = getLocationElement(api, location, location_type = l_type)  
        
        for element in search_result:
            if location == element["name"]:
                e_key = element["key"]
                e_name = element["name"]
                e_type = element["type"]

                locations_dict[e_name] = {"key": e_key, "type": e_type}

                print (str(e_key) + "\t" + str(e_name) + "\t" + str(e_type))
            
            break        
            
    return locations_dict

## INITIALIZE YOUR ACCOUNT

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv("C:/Users/coimbravieira/token_fb.txt")

In [11]:
#COMPLETE YOUT TOKEN
token = "YOUR TOKEN HERE"
#COMPLETE YOUR ACT ID
act_id = "YOUR ACT ID HERE"
secret = "-"
  
facebook_api, account = get_ad_account(token, act_id, secret)

In [12]:
#COMPLETE YOUT TOKEN
token = str(df.iloc[3,1])
#COMPLETE YOUR ACT ID
act_id = str(df.iloc[0,1])
secret = "-"
  
facebook_api, account = get_ad_account(token, act_id, secret)

## EXAMPLES:

### Collecting some interests id

In [13]:
#In my case, the names are in Portuguese
list_of_interests_to_search = ['feijoada', 'pretzel', 'pizza']

In [14]:
print("INTERESTS TO SEARCH:")
interests_dict = testInterestSearch(facebook_api, list_of_interests_to_search)

INTERESTS TO SEARCH:
interest_id 	 name 	 audience_size 	 type
6003130694636	pretzel	6137568	Food and drink
6003668857118	pizza	441634838	Food and drink


In [15]:
interests_dict

{'pretzel': {'id': '6003130694636',
  'audience': 6137568,
  'type': 'Food and drink'},
 'pizza': {'id': '6003668857118',
  'audience': 441634838,
  'type': 'Food and drink'}}

### Collecting some locations key

In [16]:
#In my case, the names are in Portuguese
list_of_locations_to_search = [('Brazil', 'country'), ('Germany', 'country'), ('Italy', 'country')]

In [ ]:
print("LOCATIONS TO SEARCH:")
locations_dict = testSearchForLocationCode(facebook_api, list_of_locations_to_search)

LOCATIONS TO SEARCH:
key 	 name 	 location_type


In [ ]:
locations_dict

### Defining your target
#### In this example we want all the Facebook users who are between 13 and 65+ years old.

In [ ]:
targeting_spec = {
    "publisher_platforms": ["facebook"], #facebook, instagram
    "facebook_positions": ["feed"], #feed, right_hand_column, instream_video
    "age_min": 13, #The min age default is 18, so I've changed to 13. The max age default is 65+
    "excluded_publisher_categories": [],
    "excluded_publisher_list_ids": [],
    "user_device": [],
    "excluded_user_device": [],
    "user_os": [],
    "wireless_carrier": [],
    'behaviors': [],
    'interests': [],
    "flexible_spec": []        
}

### Collecting the audience in each country

In [ ]:
for location in locations_dict:
    #Adding the information about the location of our target
    if locations_dict[location]["type"] == "country":
        targeting_spec["geo_locations"] = {"countries":[locations_dict[location]["key"]]}
    else:
        targeting_spec["geo_locations"] = {"regions":[{"key":locations_dict[location]["key"]}]}
    
    #We will not specify interests because we are interested on the country audience
    targeting_spec["interests"] = []
        
    audience_size = make_request(account,targeting_spec)
    
    print ("Population in " + str(location) + " = " + str(audience_size))
    
    locations_dict[location]["audience"] = audience_size

In [ ]:
locations_dict

### Collecting the audience in each interests for each country

In [ ]:
dict_info = dict()

for location in locations_dict:
    
    dict_info[location] = dict()
    
    #Adding the information about the location of our target
    if locations_dict[location]["type"] == "country":
        targeting_spec["geo_locations"] = {"countries":[locations_dict[location]["key"]]}
    else:
        targeting_spec["geo_locations"] = {"regions":[{"key":locations_dict[location]["key"]}]}

    for interest in interests_dict:
        #Adding the information about the interest we are looking for
        targeting_spec["interests"] = [interests_dict[interest]["id"]]

        audience_size = make_request(account,targeting_spec)
        
        print ("Population in " + str(location) + " interested in " + str(interest) + " = " + str(audience_size))
        
        #Wait 2 seconds between requests
        time.sleep(2)

        dict_info[location][interest] = (audience_size, float(audience_size/float(locations_dict[location]["audience"])))

In [ ]:
dict_info

### Plots

In [ ]:
x_int = range(len(interests_dict))
x = interests_dict.keys()

for location in dict_info:
    y, y_norm = zip(*dict_info[location].values())
    
    plt.bar(range(len(dict_info[location])), y_norm)
    
    plt.title(location)
    plt.xticks(x_int, x)
    plt.xlabel("interests")
    plt.ylim((0,0.5))
    plt.ylabel("audience_ic/audience_c")
    
    plt.show()